# 转换模型

TODO: 没有动态的维度支持, 有个报错, 只有将 batch_size 设置为 1

```bash
[01/04/2023-04:21:30] [TRT] [E] 10: [optimizer.cpp::computeCosts::2011] Error Code 10: Internal Error (Could not find any implementation for node {ForeignNode[bert.embeddings.position_embeddings.weight.../cls/predictions/decoder/Add]}.)
[01/04/2023-04:21:30] [TRT] [E] 2: [builder.cpp::buildSerializedNetwork::609] Error Code 2: Internal Error (Assertion enginePtr != nullptr failed. )
```

!polygraphy convert ./onnx/model.onnx --convert-to trt -o ./onnx/model.trt --model-type onnx \
	--trt-min-shapes input_ids:[1,128] attention_mask:[1,128] token_type_ids:[1,128] \
	--trt-opt-shapes input_ids:[1,128] attention_mask:[1,128] token_type_ids:[1,128] \
	--trt-max-shapes input_ids:[1,128] attention_mask:[1,128] token_type_ids:[1,128] 

https://github.com/NVIDIA/TensorRT/tree/main/tools/Polygraphy#command-line-toolkit

polygraphy 工具在 TensorRT 仓库下, 如果你安装了 tensorrt 的 python 包, 应该就能找到

/opt/conda/bin/polygraphy

In [ ]:
import tensorrt as trt
from tensorrt.tensorrt import Logger, Runtime

import os
import sys
sys.path.append("./transformer-deploy/src/")

from transformer_deploy.backends.trt_utils import build_engine, load_engine

In [ ]:
trt_logger: Logger = trt.Logger(trt.Logger.ERROR)
runtime: Runtime = trt.Runtime(trt_logger)
profile_index = 0
max_seq_len = 128
batch_size = 4

In [ ]:
tensorrt_model = load_engine(
    runtime=runtime,
    engine_file_path="./onnx/model.trt",
    profile_index=profile_index,
)

In [ ]:
import torch
import numpy
from transformers import BertTokenizer
enc = BertTokenizer.from_pretrained('bert-base-uncased')

masked_sentences = [
    # 'Paris is the [MASK] of France.', 
    # 'The primary [MASK] of the United States is English.', 
    'A baseball game consists of at least nine [MASK].', 
    # 'Topology is a branch of [MASK] concerned with the properties of geometric objects that remain unchanged under continuous transformations.'
]
pos_masks = [4, 3, 9, 6]

inputs = enc(masked_sentences, return_tensors="pt", padding='max_length', max_length=128, truncation=True)
print(inputs.keys())

result = tensorrt_model(inputs)
print(type(result))
print(len(result))
print(result[0].shape)
print(result[0])

In [ ]:
# TODO: 还是不行, 输出的结果不对. 好像每个输出的结果都是一样的